Dataset :  
RAW_recipes [here](https://drive.google.com/file/d/1vR-pUqa_RMeu8xpGCfZ15F_w2LgPP1pQ/view?usp=sharing)  
RAW_interactions [here](https://drive.google.com/file/d/13Wj5srcVPsZL9kl3otx8LaXzud4a06G7/view?usp=sharing)  
interactions_train [here](https://drive.google.com/file/d/18mHDrj6pQP2VIZzs2lUZuR5etIEx6vVz/view?usp=sharing)  
interactions_test [here](https://drive.google.com/file/d/1rCjcLbZ7IURflA0rE_ZEu-21i88ZgyqM/view?usp=sharing)  


In [ ]:
!pip install tensorflow_recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

from typing import Dict, Text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_interact = pd.read_csv("/content/drive/MyDrive/Capstone/RAW_interactions.csv")
data_recipes = pd.read_csv("/content/drive/MyDrive/Capstone/RAW_recipes.csv")

interact_train = pd.read_csv("/content/drive/MyDrive/Capstone/interactions_train.csv")
interact_test = pd.read_csv("/content/drive/MyDrive/Capstone/interactions_test.csv")

In [ ]:
pd.set_option('display.max_columns',28)
data_interact.head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [ ]:
pd.set_option('display.max_columns',28)
data_recipes.head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
nutrition_columns = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']
data_recipes[nutrition_columns] = pd.DataFrame(data_recipes['nutrition'].apply(eval).tolist(), index=data_recipes.index)
data_recipes.drop('nutrition', axis=1, inplace=True)

# Display the modified DataFrame
print("\nDataFrame with Extracted Nutrition Information:")


DataFrame with Extracted Nutrition Information:


In [ ]:
pd.set_option('display.max_columns',28)
data_recipes.head(5)

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [ ]:
data_interact = data_interact.astype({'user_id': 'string', 'recipe_id':'string'})
interact_train = interact_train.astype({'user_id': 'string', 'recipe_id':'string'})
interact_test = interact_test.astype({'user_id': 'string', 'recipe_id':'string'})

In [ ]:
user_id = data_interact.user_id.unique()
recipe_id = data_interact.recipe_id.unique()

In [ ]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=user_id, mask_token=None),
                                        # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(user_id)+1, embedding_dimension)
                                    ])

        self.product_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=recipe_id, mask_token=None),
                                    # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(recipe_id)+1, embedding_dimension)
                                    ])
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation="relu"),
                            tf.keras.layers.Dense(64,  activation="relu"),
                            tf.keras.layers.Dense(1)
                              ])

    def call(self, userId, foodId):
        user_embeddings  = self.user_embeddings (userId)
        food_embeddings = self.product_embeddings(foodId)
        return self.ratings(tf.concat([user_embeddings, food_embeddings], axis=1))



In [ ]:
# Build a model.
class FoodModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer   = tfrs.tasks.Ranking(
                                                    loss    =  tf.keras.losses.MeanSquaredError(),
                                                    metrics = [tf.keras.metrics.RootMeanSquaredError()])


    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userID"], features["foodID"]  )

        return self.task( labels=features["rating"], predictions=rating_predictions)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(interact_train.user_id.values, tf.string),
    "foodID":tf.cast(interact_train.recipe_id.values, tf.string),
    "rating":tf.cast(interact_train.rating.values, tf.float32)
})

test_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(interact_test.user_id.values, tf.string),
    "foodID":tf.cast(interact_test.recipe_id.values, tf.string),
    "rating":tf.cast(interact_test.rating.values, tf.float32)
})

In [ ]:
tf.random.set_seed(42)

train_data = train_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

In [ ]:
model = FoodModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001))
cached_train = train_data.shuffle(100_000).batch(8192).cache()
cached_test = test_data.batch(4096).cache()
model.fit(cached_train, epochs=10)

Epoch 1/10
86/86 [==============================] - 12s 125ms/step - root_mean_squared_error: 4.4926 - loss: 20.1561 - regularization_loss: 0.0000e+00 - total_loss: 20.1561
Epoch 2/10
86/86 [==============================] - 5s 57ms/step - root_mean_squared_error: 4.1514 - loss: 17.2033 - regularization_loss: 0.0000e+00 - total_loss: 17.2033
Epoch 3/10
86/86 [==============================] - 4s 46ms/step - root_mean_squared_error: 3.7355 - loss: 13.9212 - regularization_loss: 0.0000e+00 - total_loss: 13.9212
Epoch 4/10
86/86 [==============================] - 4s 51ms/step - root_mean_squared_error: 3.2542 - loss: 10.5599 - regularization_loss: 0.0000e+00 - total_loss: 10.5599
Epoch 5/10
86/86 [==============================] - 6s 72ms/step - root_mean_squared_error: 2.7512 - loss: 7.5447 - regularization_loss: 0.0000e+00 - total_loss: 7.5447
Epoch 6/10
86/86 [==============================] - 4s 45ms/step - root_mean_squared_error: 2.2706 - loss: 5.1380 - regularization_loss: 0.0000e+

In [ ]:
user_rand = user_id[200]
test_rating = {}
for m in test_data.take(10):
    test_rating[m["foodID"].numpy()]=RankingModel()(tf.convert_to_tensor([user_rand]),tf.convert_to_tensor([m["foodID"]]))

In [ ]:
print("Top 10 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(data_recipes.loc[data_recipes['id'] == int(m.decode())]['name'].item())

Top 10 recommended products for User 450004: 
peanut butter spirals
breakfast biscuit sandwiches
skinny style chocolatey hot cocoa
azumaya pot stickers
ww 3 points   grilled beef fajitas
ranch egg salad croissant sandwiches
strawberry fudge
sweet and sour pork ribs
lemon herb chicken w mushrooms low carb
blueberry pancake syrup   low carb


In [ ]:
# Input user id
user_id_to_recommend = "102602"

# Get the user's unique index in the vocabulary
user_index = np.where(user_id == user_id_to_recommend)[0]

# If the user is not in the vocabulary, you can handle this case accordingly
if user_index.size == 0:
    print(f"User with id {user_id_to_recommend} not found.")
else:
    user_index = user_index[0]# Extract the scalar value
    # Generate recommendations for the specified user
    user_embeddings = model.ranking_model.user_embeddings(tf.constant([user_id[user_index]]))
    all_food_ids = tf.constant(recipe_id, dtype=tf.string)
    food_embeddings = model.ranking_model.product_embeddings(all_food_ids)
    scores = tf.reduce_sum(user_embeddings * food_embeddings, axis=1)

    # Get the top 10 food indices based on scores
    top_food_indices = tf.argsort(scores, direction='DESCENDING')[:10]

    # Display the recommended food names
    print(f"Top 10 recommended products for User {user_id_to_recommend}:")
    for food_index in top_food_indices.numpy():
        food_name = data_recipes.loc[data_recipes['id'] == int(recipe_id[food_index])]['name'].item()
        print(food_name)

Top 10 recommended products for User 102602:
grilled banana and nutella panini
spaghettini with roasted garlic
fooll mudammes  fava bean egyptian breakfast
veggie brunch
sausage velveeta slow cooker dip
quick cherry turnovers
aimee s creamed tuna luncheon
linguine with pancetta and arugula
rice  krispies bars loaded
lemon chicken bow tie pasta
